In [23]:
import cohere
from getpass import getpass
import os
import requests
from flask import Flask, request, jsonify
# from flask_cors import CORS
from fastapi import FastAPI

# Get Cohere API key securely
COHERE_API_KEY = getpass("Enter your Cohere API key: ")

# Initialize the Cohere client
os.environ["COHERE_API_KEY"] = COHERE_API_KEY

# FastAPI endpoint
FASTAPI_URL = "http://127.0.0.1:8000/process-text/"

# FastAPI endpoint for fetching the array
FASTAPI_ARRAY_URL = "http://127.0.0.1:8000/get-array/"

# Initialize Flask app
app = FastAPI()

# Enable CORS for the Flask app


In [24]:
from langchain.llms import Cohere

llm = Cohere(cohere_api_key=COHERE_API_KEY)

def get_prompt_from_api(input_text):
    try:
        response = requests.post(
            FASTAPI_URL,
            json={"text": input_text},
            headers={"Content-Type": "application/json"}
        )
        response.raise_for_status()
        return response.json().get("processed_text", "")
    except Exception as e:
        print("Error fetching prompt from API:", e)
        return None


@app.route("/get-ai-response", methods=["GET"])
def get_ai_response():
    try:
        # Fetch the array from FastAPI
        response = requests.get(FASTAPI_ARRAY_URL, headers={"Content-Type": "application/json"})
        response.raise_for_status()
        text_array = response.json().get("array", [])

        if not text_array:
            return jsonify({"error": "Array is empty"}), 400

        # Get the last element of the array
        last_text = text_array[-1]

        # Get response from Cohere LLM
        try:
            llm_response = llm.invoke(last_text)
            if not llm_response:
                raise ValueError("LLM returned an empty response")
        except Exception as e:
            return jsonify({"error": f"Error processing LLM response: {str(e)}"}), 500

        return jsonify({"response": llm_response}), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500

In [22]:
# Run the Flask app
if __name__ == "__main__":
    app.run(port=8000)

 * Serving Flask app '__main__'
 * Debug mode: off
 * Debug mode: off


 * Running on http://127.0.0.1:8000
Press CTRL+C to quit

 * Running on http://127.0.0.1:8000
Press CTRL+C to quit
